In [ ]:
import pandas as pd
import numpy as np
import pulp

# The Transportation Problem

<img src="The Transportation Problem image 01.png">

In this scenario, the comapny SiliconTat AS has three plants from where they produce silicon. The silicon is used by their customers to produce e.g. solar panels, circuit boards and even as a texture modifier to make a material waterproof. Let us assume that SiliconTat AS has five customers who are all interesten in their product. However, the plants have a maximum production capacity of what they can deliver, and the customers have a maximum demand. Further, the customers are located at very different locations, and the shipping cost to each of them vary greatly. We will in this notebook find the optimal number silicium to ship to each customer to minimize the cost of shipping.

Note that in this case total production capacity exceeds total demand.

|    &nbsp;   |    Customer 1 |     Customer 2 |     Customer 3 |     Customer 4 |     Customer 5 | PRODUCTION CAPACITY |
| ----------- | ------------- | -------------- | -------------- | -------------- | -------------- | ------------------- |
| **Plant 1** |            7  |              6 |              9 |             12 |             18 |          **32 000** |
| **Plant 2** |            6  |              4 |              8 |              9 |             15 |          **19 000** |
| **Plant 3** |           17  |             11 |              8 |              7 |              7 |          **21 000** |
|  **DEMAND** |    **18 000**  |    **11 000**  |      **9 000** |     **12 000** |     **15 000** |              &nbsp; |

## Prepare the data

In [ ]:
data = [
    [7, 6, 9, 12, 18, 32000],
    [6, 4, 8, 9, 15, 19000],
    [17, 11, 8, 7, 7, 21000],
    [18000, 11000, 9000, 12000, 15000]]
columns = ['Customer 1', 'Customer 2', 'Customer 3', 'Customer 4', 'Customer 5', 'PRODUCTION CAPACITY']
index = ['Plant 1', 'Plant 2', 'Plant 3', 'DEMAND']

df = pd.DataFrame(data=data, columns=columns, index=index)
df.head()

## Create the variables
We wil define the following variables to model the network:

$$
\begin{align*}
x_{p,c} &= \textrm{number of units sent from plant } j \textrm{ to customer } j \\
\end{align*}
$$

In [ ]:
Plants = ['Plant 1', 'Plant 2', 'Plant 3']
Customers = ['Customer 1', 'Customer 2', 'Customer 3', 'Customer 4', 'Customer 5']

x = pulp.LpVariable.dicts("units",
                          ((p, c) for p in Plants for c in Customers),
                          lowBound=0,
                          cat='Integer')
x

## Initiate an empty LP Problem

In [ ]:
prob = pulp.LpProblem("TransportationProblem", pulp.LpMinimize)

## Create constraints

### Ensure that the plants can not exceed their production capacity

In [ ]:
for p in Plants:
    prob += pulp.lpSum([x[p,c] for c in Customers]) <= df.loc[p,'PRODUCTION CAPACITY'], f"Production capacity for {p}"

### Ensure the maximum demand for each customer is met

In [ ]:
for c in Customers:
    prob += pulp.lpSum([x[p,c] for p in Plants]) == df.loc['DEMAND',c], f"Demand from {c}"

## Create the objective function
Since the total cost will be the sum of all decision variables $x_{i,j}$ multiplied with its corresponding cost, this is objective we would like to minimize, and will be our objective function

In [ ]:
prob += pulp.lpSum(df.loc[Plants, Customers].to_numpy() * np.array([[x[p,c] for c in Customers] for p in Plants]))

In [ ]:
prob

## Find the optimal solution

In [ ]:
prob.solve()
status = pulp.LpStatus[prob.status]
obj_value = prob.objective.value()

print(f"The solver found a solution that is {status}, which gives a shippment cost of {obj_value:.2f} NOK")

In [ ]:
results = pd.DataFrame(data=[[x[p,c].value() for c in Customers] for p in Plants], index=Plants, columns=Customers)
results = results.append([pd.Series(results[Customers].sum(), name='SUM OF PRODUCTION')])
results['SUM OF PRODUCTION'] = results.iloc[:-1].sum(axis=1)


print("We can investigate the number of units to ship from every pair of plants and customers in the following table")
results